In [3]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#  Data Preprocessing

In [5]:
anime_df = pd.read_csv('anime.csv')
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [6]:
anime_df.shape

(12294, 7)

In [10]:
anime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [12]:
anime_df.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [14]:
anime_df.dtypes

anime_id      int64
name         object
genre        object
type         object
episodes     object
rating      float64
members       int64
dtype: object

In [16]:
# Handle missing values
anime_df['genre'].fillna("Unknown", inplace=True)
anime_df['type'].fillna("Unknown", inplace=True)
anime_df['rating'].fillna(anime_df['rating'].mean(), inplace=True)

In [18]:
anime_df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [20]:
# One-hot encode 'genre' and 'type'
anime_df['genre'] = anime_df['genre'].apply(lambda x: x.split(', ') if x != "Unknown" else ["Unknown"])

In [22]:
mlb = MultiLabelBinarizer()
mlb

MultiLabelBinarizer()

In [24]:
genre_encoded = mlb.fit_transform(anime_df['genre'])
genre_encoded

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [26]:
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)
genre_df.head()

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# encode type colum to numeric column.

In [30]:
type_encoded = pd.get_dummies(anime_df['type'], prefix='type')

In [32]:
# Normalize numerical features
scaler = MinMaxScaler()

In [34]:
anime_df[['rating', 'members']] = scaler.fit_transform(anime_df[['rating', 'members']])

In [36]:
# Combine features
anime_df = pd.concat([anime_df, genre_df, type_encoded], axis=1)

In [38]:
# Step 2: Feature Extraction
features = anime_df.drop(columns=['anime_id', 'name', 'genre', 'type', 'episodes']).values
features

array([[0.9243697478991597, 0.19787220192679442, 0, ..., False, False,
        False],
       [0.9111644657863145, 0.7827701023362974, 1, ..., False, True,
        False],
       [0.9099639855942379, 0.11268926691862803, 1, ..., False, True,
        False],
       ...,
       [0.38535414165666265, 0.00021106368205524737, 0, ..., False,
        False, False],
       [0.397358943577431, 0.00016766741097846755, 0, ..., False, False,
        False],
       [0.4549819927971189, 0.00013512020767088265, 0, ..., False, False,
        False]], dtype=object)

#  Recommendation System

In [41]:
# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(features)

In [51]:
# Create a name-to-index lookup dictionary
name_to_index = pd.Series(anime_df.index, index=anime_df['name']).to_dict()

In [53]:


def get_recommendations(anime_name, similarity_matrix, top_n=5):
    if anime_name not in name_to_index:
        print(f"Anime '{anime_name}' not found in the dataset.")
        return[]

    # get index of the anime
    anime_index=name_to_index[anime_name]
    
    # get the similarty scores for the given anime 
    similarity_scores = list(enumerate(similarity_matrix[anime_index]))
                                       
    # Sort by similarity scores, ignore the first entry (the anime itself)
    sorted_similar_anime = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    
    
      # Get recommended anime names
    recommended_anime = [anime_df.iloc[i]['name'] for i, _ in sorted_similar_anime]
    return recommended_anime


In [84]:
anime_name = "Bleach"  # Replace with any anime name in the dataset

In [86]:
recommendations = get_recommendations(anime_name, similarity_matrix)
recommendations

['Katekyo Hitman Reborn!',
 'Code:Breaker',
 'Yozakura Quartet: Hana no Uta',
 'Tokyo ESP',
 'Yozakura Quartet']

#  Evaluation

In [60]:
#split data into training and testing
train_df, test_df = train_test_split(anime_df, test_size=0.2, random_state=42)

In [62]:
train_features = train_df.drop(columns=['anime_id', 'name', 'genre', 'type', 'episodes']).values

In [64]:
test_features = test_df.drop(columns=['anime_id', 'name', 'genre', 'type', 'episodes']).values

In [96]:
def evaluate_recommendations(anime_name, top_n=5):
    if anime_name not in name_to_index:
        print(f"Anime '{anime_name}' not found in the dataset.")
        return None
    
    # Get ground truth: Similar anime based on shared genres
    anime_index = name_to_index[anime_name]
    target_genres = set(anime_df.iloc[anime_index]['genre'])
    ground_truth_indices = anime_df[anime_df['genre'].apply(lambda x: bool(set(x).intersection(target_genres)))].index
    y_true = [1 if i in ground_truth_indices else 0 for i in range(len(anime_df))]
    
    # Get predictions
    recommended_indices = recommend_anime_indices(anime_name, similarity_matrix, top_n=top_n)
    y_pred = [1 if i in recommended_indices else 0 for i in range(len(anime_df))]
    
    # Calculate precision, recall, and F1-score
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    return precision, recall, f1




In [98]:
# Evaluate recommendations for an example anime
anime_name = "Kimi no Na wa."  # Replace with any anime name in the dataset
precision, recall, f1 = evaluate_recommendations(anime_name, top_n=5)

print(f"Evaluation for '{anime_name}':")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


Evaluation for 'Kimi no Na wa.':
Precision: 1.00
Recall: 0.00
F1-Score: 0.00


# Interiew Questions

# Question 1 : Can you explain the difference between user-based and item-based collaborative filtering?


### Both user-based and item-based collaborative filtering are techniques used in recommendation systems. The key difference lies in the perspective from which recommendations are generated.

## 1. User-Based Collaborative Filtering:
### Definition: Recommendations are made based on the similarity between users.
## How It Works:
### Identify similar users to the target user using a similarity measure (e.g., cosine similarity, Pearson correlation).
### Recommend items that the similar users have liked but the target user has not interacted with.
## Example:
### If User A and User B have similar preferences, and User B likes a movie that User A has not seen, that movie is recommended to User A.
## Advantages:
### Intuitive and works well when there are many users with overlapping preferences.
## Disadvantages:
### Struggles with cold start for new users (insufficient interaction data).
### Performance may degrade with large numbers of users.


## Item-Based Collaborative Filtering
### Definition: Recommendations are made based on the similarity between items.
## How It Works:
### Identify similar items to those that the target user has interacted with.
### ecommend items similar to those the user has liked or interacted with.
## Example:
### If a user likes Movie A, and Movie B is similar to Movie A (based on ratings from all users), Movie B is recommended.
## Advantages:
### More scalable as the number of items is typically smaller than the number of users.
### Works well for new users as long as they interact with a few items.
## Disadvantages:
### Struggles with cold start for new items with little to no interaction history.

# ____________________________

# Question 2: What is collaborative filtering, and how does it work?

## Definition
### Collaborative filtering is a technique used in recommendation systems to suggest items based on patterns of user-item interactions. It relies on the principle that users who interacted similarly with certain items in the past are likely to have similar preferences.

## Types of Collaborative Filtering:
## 1. User-Based Collaborative Filtering:
### Focuses on finding users with similar preferences or behaviors.
## 2.Item-Based Collaborative Filtering:
### Focuses on finding items that are similar based on user interactions.

## How Collaborative Filtering Works
## Steps:
## Data Collection:

### Gather user-item interaction data, typically in the form of a ratings matrix.

## Similarity Computation:

### Calculate similarity between users or items using metrics like:
### 1.Cosine Similarity: Measures the cosine of the angle between vectors.
### 2.Pearson Correlation: Measures linear correlation between vectors.
### 3.Jaccard Similarity: Measures overlap between sets.

## Prediction Generation:

### For user-based filtering: Predict the rating a user would give to an item by averaging the ratings of similar users.
### For item-based filtering: Predict the rating a user would give to an item based on the ratings the user has given to similar items.

## Recommendation:

### Recommend the top 𝑁 items with the highest predicted ratings that the user has not interacted with.

# ---- The End -----